In [2]:
import pandas as pd
pd.set_option('display.max_columns', None)
import numpy as np
from sklearn.model_selection import train_test_split
import shap
import joblib
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from imblearn.over_sampling import SMOTE, ADASYN
from sklearn.preprocessing import normalize
from sklearn.model_selection import train_test_split
import xgboost

from sklearn.preprocessing import StandardScaler, MinMaxScaler
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

from sklearn.metrics import plot_roc_curve, classification_report, accuracy_score, f1_score, recall_score, precision_score, confusion_matrix

pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.


In [2]:
df=pd.read_csv('CLV.csv')
df.head()

,Unnamed: 0,city,Customer Lifetime Value,feedback,type_of_plan,highest_level_education,Effective To Date,work_status,sex,salary_monthly,relationship_status,payment_per_month,weeks_since_claim,open_policies,Renew Offer Type,reachability,claim_amount,type_of_vehicle
0,0,Tamale,3730.751027,No,primary_plan,Bachelor,2/24/11,Employed,female,9702.413793,Married,138,138.0,1,1,contacting_agent,384.811147,2-door_car
1,1,Takoradi,9422.373469,No,extended_plan,Bachelor,1/31/11,Unemployed,female,0.000000,Single,188,56.0,8,3,contacting_agent,1131.464935,4-door_car
2,2,Cape_coast,17398.032728,No,premium_plan,Bachelor,2/19/11,Employed,female,8408.103448,Married,216,77.0,2,1,contacting_agent,566.472247,2-door_car
3,3,Accra,10321.913466,No,primary_plan,Bachelor,1/20/11,Unemployed,male,0.000000,Married,212,77.0,7,1,customer_call_centre,529.881344,SUV
4,4,Tamale,3798.484976,No,primary_plan,Bachelor,2/03/2011,Employed,male,7557.931034,Single,146,52.0,1,1,contacting_agent,138.130879,4-door_car


In [3]:
df=df.rename(columns={'feedback':'churn'})

In [4]:
df['churn'].value_counts()

No     7826
Yes    1308
Name: churn, dtype: int64

In [5]:
df['churn']=df['churn'].replace({0:'Not_churned', 1:'churned'})

In [6]:
df.head()

,Unnamed: 0,city,Customer Lifetime Value,churn,type_of_plan,highest_level_education,Effective To Date,work_status,sex,salary_monthly,relationship_status,payment_per_month,weeks_since_claim,open_policies,Renew Offer Type,reachability,claim_amount,type_of_vehicle
0,0,Tamale,3730.751027,No,primary_plan,Bachelor,2/24/11,Employed,female,9702.413793,Married,138,138.0,1,1,contacting_agent,384.811147,2-door_car
1,1,Takoradi,9422.373469,No,extended_plan,Bachelor,1/31/11,Unemployed,female,0.000000,Single,188,56.0,8,3,contacting_agent,1131.464935,4-door_car
2,2,Cape_coast,17398.032728,No,premium_plan,Bachelor,2/19/11,Employed,female,8408.103448,Married,216,77.0,2,1,contacting_agent,566.472247,2-door_car
3,3,Accra,10321.913466,No,primary_plan,Bachelor,1/20/11,Unemployed,male,0.000000,Married,212,77.0,7,1,customer_call_centre,529.881344,SUV
4,4,Tamale,3798.484976,No,primary_plan,Bachelor,2/03/2011,Employed,male,7557.931034,Single,146,52.0,1,1,contacting_agent,138.130879,4-door_car


In [7]:
df=df.drop(columns='Unnamed: 0')

In [8]:
df.to_csv('investment_churn.csv')

In [9]:
df.head()

,city,Customer Lifetime Value,churn,type_of_plan,highest_level_education,Effective To Date,work_status,sex,salary_monthly,relationship_status,payment_per_month,weeks_since_claim,open_policies,Renew Offer Type,reachability,claim_amount,type_of_vehicle
0,Tamale,3730.751027,No,primary_plan,Bachelor,2/24/11,Employed,female,9702.413793,Married,138,138.0,1,1,contacting_agent,384.811147,2-door_car
1,Takoradi,9422.373469,No,extended_plan,Bachelor,1/31/11,Unemployed,female,0.000000,Single,188,56.0,8,3,contacting_agent,1131.464935,4-door_car
2,Cape_coast,17398.032728,No,premium_plan,Bachelor,2/19/11,Employed,female,8408.103448,Married,216,77.0,2,1,contacting_agent,566.472247,2-door_car
3,Accra,10321.913466,No,primary_plan,Bachelor,1/20/11,Unemployed,male,0.000000,Married,212,77.0,7,1,customer_call_centre,529.881344,SUV
4,Tamale,3798.484976,No,primary_plan,Bachelor,2/03/2011,Employed,male,7557.931034,Single,146,52.0,1,1,contacting_agent,138.130879,4-door_car


In [28]:
df.to_csv('veh_invest.csv')

In [27]:
df=df.drop(columns=['Customer Lifetime Value','Effective To Date'])
df.head()

,city,churn,type_of_plan,highest_level_education,work_status,sex,salary_monthly,relationship_status,payment_per_month,weeks_since_claim,open_policies,Renew Offer Type,reachability,claim_amount,type_of_vehicle
0,Tamale,No,primary_plan,Bachelor,Employed,female,9702.413793,Married,138,138.0,1,1,contacting_agent,384.811147,2-door_car
1,Takoradi,No,extended_plan,Bachelor,Unemployed,female,0.000000,Single,188,56.0,8,3,contacting_agent,1131.464935,4-door_car
2,Cape_coast,No,premium_plan,Bachelor,Employed,female,8408.103448,Married,216,77.0,2,1,contacting_agent,566.472247,2-door_car
3,Accra,No,primary_plan,Bachelor,Unemployed,male,0.000000,Married,212,77.0,7,1,customer_call_centre,529.881344,SUV
4,Tamale,No,primary_plan,Bachelor,Employed,male,7557.931034,Single,146,52.0,1,1,contacting_agent,138.130879,4-door_car


In [19]:
df=pd.read_csv('veh_invest.csv')

In [20]:
df.head()

,Unnamed: 0,city,churn,type_of_plan,highest_level_education,work_status,sex,salary_monthly,relationship_status,payment_per_month,weeks_since_claim,open_policies,Renew Offer Type,reachability,claim_amount,type_of_vehicle
0,0,Tamale,No,primary_plan,Bachelor,Employed,female,9702.413793,Married,138,138.0,1,1,contacting_agent,384.811147,2-door_car
1,1,Takoradi,No,extended_plan,Bachelor,Unemployed,female,0.000000,Single,188,56.0,8,3,contacting_agent,1131.464935,4-door_car
2,2,Cape_coast,No,premium_plan,Bachelor,Employed,female,8408.103448,Married,216,77.0,2,1,contacting_agent,566.472247,2-door_car
3,3,Accra,No,primary_plan,Bachelor,Unemployed,male,0.000000,Married,212,77.0,7,1,customer_call_centre,529.881344,SUV
4,4,Tamale,No,primary_plan,Bachelor,Employed,male,7557.931034,Single,146,52.0,1,1,contacting_agent,138.130879,4-door_car


In [21]:
df=df.drop(columns='Unnamed: 0')

In [22]:
df_ohe=pd.get_dummies(df.drop(columns=['payment_per_month','weeks_since_claim','open_policies','Renew Offer Type','claim_amount','salary_monthly','churn']))

In [23]:
df_ohe.head()

,city_Accra,city_Cape_coast,city_Kumasi,city_Takoradi,city_Tamale,type_of_plan_extended_plan,type_of_plan_premium_plan,type_of_plan_primary_plan,highest_level_education_Bachelor,highest_level_education_College,highest_level_education_Doctor,highest_level_education_High School or Below,highest_level_education_Master,work_status_Disabled,work_status_Employed,work_status_Medical Leave,work_status_Retired,work_status_Unemployed,sex_female,sex_male,relationship_status_Divorced,relationship_status_Married,relationship_status_Single,reachability_contacting_agent,reachability_contacting_branch_directly,reachability_customer_call_centre,reachability_via_the_web,type_of_vehicle_2-door_car,type_of_vehicle_4-door_car,type_of_vehicle_SUV,type_of_vehicle_Sports Car,type_of_vehicle_luxirious_car,type_of_vehicle_luxurious_suv
0,0,0,0,0,1,0,0,1,1,0,0,0,0,0,1,0,0,0,1,0,0,1,0,1,0,0,0,1,0,0,0,0,0
1,0,0,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0,1,1,0,0,0,1,1,0,0,0,0,1,0,0,0,0
2,0,1,0,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,1,0,0,1,0,1,0,0,0,1,0,0,0,0,0
3,1,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,1,0,1,0,1,0,0,0,1,0,0,0,1,0,0,0
4,0,0,0,0,1,0,0,1,1,0,0,0,0,0,1,0,0,0,0,1,0,0,1,1,0,0,0,0,1,0,0,0,0


In [24]:
df_left=df[['payment_per_month','weeks_since_claim','open_policies','Renew Offer Type','claim_amount','salary_monthly','churn']]

In [25]:
df_new=pd.concat([df_left, df_ohe],axis=1)

In [26]:
df_new.head()

,payment_per_month,weeks_since_claim,open_policies,Renew Offer Type,claim_amount,salary_monthly,churn,city_Accra,city_Cape_coast,city_Kumasi,city_Takoradi,city_Tamale,type_of_plan_extended_plan,type_of_plan_premium_plan,type_of_plan_primary_plan,highest_level_education_Bachelor,highest_level_education_College,highest_level_education_Doctor,highest_level_education_High School or Below,highest_level_education_Master,work_status_Disabled,work_status_Employed,work_status_Medical Leave,work_status_Retired,work_status_Unemployed,sex_female,sex_male,relationship_status_Divorced,relationship_status_Married,relationship_status_Single,reachability_contacting_agent,reachability_contacting_branch_directly,reachability_customer_call_centre,reachability_via_the_web,type_of_vehicle_2-door_car,type_of_vehicle_4-door_car,type_of_vehicle_SUV,type_of_vehicle_Sports Car,type_of_vehicle_luxirious_car,type_of_vehicle_luxurious_suv
0,138,138.0,1,1,384.811147,9702.413793,No,0,0,0,0,1,0,0,1,1,0,0,0,0,0,1,0,0,0,1,0,0,1,0,1,0,0,0,1,0,0,0,0,0
1,188,56.0,8,3,1131.464935,0.000000,No,0,0,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0,1,1,0,0,0,1,1,0,0,0,0,1,0,0,0,0
2,216,77.0,2,1,566.472247,8408.103448,No,0,1,0,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,1,0,0,1,0,1,0,0,0,1,0,0,0,0,0
3,212,77.0,7,1,529.881344,0.000000,No,1,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,1,0,1,0,1,0,0,0,1,0,0,0,1,0,0,0
4,146,52.0,1,1,138.130879,7557.931034,No,0,0,0,0,1,0,0,1,1,0,0,0,0,0,1,0,0,0,0,1,0,0,1,1,0,0,0,0,1,0,0,0,0


In [27]:
#df_new['churn']=df_new['churn'].map({'No':0,'Yes':1})

In [28]:
df_new['churn']=df_new['churn'].replace({'No':0,'Yes':1})

In [29]:
df_new['churn'].value_counts(normalize=True)

0    0.856799
1    0.143201
Name: churn, dtype: float64

In [30]:
X=df_new.drop(columns='churn')
y=df_new['churn']

In [31]:
X_train,X_valid,y_train,y_valid=train_test_split(X,y, test_size=0.2, random_state=0)

In [32]:
##baseline
model=LogisticRegression(solver='liblinear')
model.fit(X_train, y_train)
pred= model.predict(X_valid)
print('accuracy:',accuracy_score(pred, y_valid))
print('f1:',f1_score(pred, y_valid,zero_division=1))
print('recall:',recall_score(pred, y_valid,zero_division=1))
print('precision:',precision_score(pred, y_valid,zero_division=1))
confusion_matrix(pred, y_valid)

accuracy: 0.8631636562671046
f1: 0.1987179487179487
recall: 0.7380952380952381
precision: 0.11481481481481481


array([[1546,  239],
       [  11,   31]])

In [33]:
#balancing using adasyn
ada = SMOTE(random_state=0)
X_res, y_res = ada.fit_resample(X, y)
X_train_res,X_valid_res,y_train_res,y_valid_res=train_test_split(X_res,y_res, test_size=0.2, random_state=0)
##baseline
model=LogisticRegression(solver='liblinear')
model.fit(X_train_res, y_train_res)
pred= model.predict(X_valid_res)
print('accuracy:',accuracy_score(pred, y_valid_res))
print('f1:',f1_score(pred, y_valid_res,zero_division=1))
print('recall:',recall_score(pred, y_valid_res,zero_division=1))
print('precision:',precision_score(pred, y_valid_res,zero_division=1))
confusion_matrix(pred, y_valid_res)

accuracy: 0.8221015649952091
f1: 0.7981152591518667
recall: 0.9499568593615185
precision: 0.688125


array([[1473,  499],
       [  58, 1101]])

In [34]:
##with smote and scaling
#Normalize the data
scaler = MinMaxScaler()
X_train_norm = scaler.fit_transform(X_train_res)
X_valid_norm = scaler.transform(X_valid_res)
model=LogisticRegression(solver='liblinear')
model.fit(X_train_norm, y_train_res)
pred= model.predict(X_valid_norm)
print('accuracy:',accuracy_score(pred, y_valid_res))
print('f1:',f1_score(pred, y_valid_res,zero_division=1))
print('recall:',recall_score(pred, y_valid_res,zero_division=1))
print('precision:',precision_score(pred, y_valid_res,zero_division=1))
confusion_matrix(pred, y_valid_res)

accuracy: 0.8233791121047589
f1: 0.7992740471869328
recall: 0.9532467532467532
precision: 0.688125


array([[1477,  499],
       [  54, 1101]])